In [1]:
import mne
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.stats import skew, kurtosis
import pandas as pd
import itertools
from scipy.signal import welch
from scipy.stats import entropy
from pyentrp import entropy as ent
import numba
from numba import jit



In [3]:
def eeg_to_csv(edf_file_path, channels, bands, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Prepare a list to collect the data
    all_data = []
    
    # For each channel
    for channel in channels:
        channel_data = {}
        channel_data['Channel'] = channel
        
        # Extract the EEG data for each frequency band and compute the metrics
        for band in bands:
            band_data = extract_data_by_band(edf_data, channel, band)
            
            # Compute metrics (we'll add more metrics here if necessary)
            channel_data[f"{band}_mean"] = np.mean(band_data)
            channel_data[f"{band}_std"] = np.std(band_data)
            channel_data[f"{band}_skew"] = skew(band_data)
            channel_data[f"{band}_kurtosis"] = kurtosis(band_data)
            channel_data[f"{band}_perm_entropy"] = permutation_entropy(band_data)
            channel_data[f"{band}_higuchi_fd"] = higuchi_fd(band_data)
            
        # Add the channel data to the main data list
        all_data.append(channel_data)
    
    # Convert the data list to a DataFrame and save it to a CSV file
    df = pd.DataFrame(all_data)
    df.to_csv(output_csv_path, index=False)
    return output_csv_path

# Let's redefine and integrate the functions from the notebook into our current environment

def extract_data_by_band(edf_data, channel_name, band_name):
    bands = {
        "delta": (2, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "low-beta": (13, 20)
    }
    l_freq, h_freq = bands[band_name]
    edf_data_filtered = edf_data.copy()
    edf_data_filtered.filter(l_freq=l_freq, h_freq=h_freq, picks=[channel_name], verbose=False)
    channel_data, _ = edf_data_filtered[edf_data_filtered.ch_names.index(channel_name)]
    return channel_data[0]

# Redefining the required functions

def permutation_entropy(signal, order=3, delay=1, normalize=True):
    n = len(signal)
    L = []
    x = []
    for k in range(1, order + 1):
        Lk = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int((n - m) / k)):
                Lmk += abs(signal[m + i * k] - signal[m + i * k - k])
            Lmk = Lmk * (n - 1) / (int((n - m) / float(k)) * k)
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(1.0 / k), 1])
    (p, _, _, _) = np.linalg.lstsq(x, L, rcond=None)
    return p[0]

def higuchi_fd(data, k_max=10):
    n = len(data)
    L = []
    x = []
    for k in range(1, k_max + 1):
        Lk = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int((n - m) / k)):
                Lmk += abs(data[m + i * k] - data[m + i * k - k])
            Lmk = Lmk * (n - 1) / (int((n - m) / float(k)) * k)
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(1.0 / k), 1])
    (p, _, _, _) = np.linalg.lstsq(x, L, rcond=None)
    return p[0]

def extract_microstate_features(assignments):
    """Extract microstate features from a sequence of microstate assignments."""
    # Compute microstate durations
    durations = [len(list(g)) for _, g in itertools.groupby(assignments)]
    
    # Compute microstate occurrences
    occurrences = [sum(1 for _ in g) for _, g in itertools.groupby(assignments)]
    
    # Compute microstate contributions
    contributions = [occurrence * duration for occurrence, duration in zip(occurrences, durations)]
    
    # Compute mean duration and mean occurrence
    mean_duration = np.mean(durations)
    mean_occurrence = np.mean(occurrences)
    
    return durations, occurrences, contributions, mean_duration, mean_occurrence

def compute_mean_gfp(data, assignments, gfp):
    """Compute mean GFP during each microstate class."""
    mean_gfps = []
    unique_states = np.unique(assignments)
    for state in unique_states:
        mean_gfp = np.mean([gfp[i] for i in range(len(gfp)) if assignments[i] == state])
        mean_gfps.append(mean_gfp)
    return mean_gfps
    
def zero_crossing_rate(data):
    # Compute the zero-crossing rate of a signal.
    return ((data[:-1] * data[1:]) < 0).sum() / len(data)

def shannon_entropy(data):
    # """Compute the Shannon entropy of a signal."""
    p_data = data / data.sum()
    return entropy(p_data)

def katz_fd(data):
    # """Compute Katz fractal dimension of a signal."""
    n = len(data)
    L = np.abs(data - data[0]).sum()
    d = np.abs(data - data[0]).max()
    return np.log10(n) / (np.log10(d/L) + np.log10(n))

def sample_entropy(data, m=2, r=0.2):
    # """Compute the sample entropy of a signal."""
    N = len(data)
    A = np.sum([np.sum(np.abs(data[i:i+m] - data[j:j+m]) < r) for i in range(N - m) for j in range(i, N - m)]) - N - m + 1
    B = np.sum([np.sum(np.abs(data[i:i+m+1] - data[j:j+m+1]) < r) for i in range(N - m - 1) for j in range(i, N - m - 1)])
    return -np.log(A / B)

def approximate_entropy(data, m=2, r=0.2):
    # """Compute the approximate entropy of a signal."""
    N = len(data)
    phi_m = np.sum([np.sum(np.abs(data[i:i+m] - data[j:j+m]) < r) for i in range(N - m) for j in range(i, N - m)]) / (N - m + 1)
    phi_m1 = np.sum([np.sum(np.abs(data[i:i+m+1] - data[j:j+m+1]) < r) for i in range(N - m - 1) for j in range(i, N - m - 1)]) / (N - m)
    return phi_m - phi_m1

def hjorth_parameter(data):
    # """Compute the Hjorth parameter of a signal."""
    diff_data = np.diff(data)
    diff_diff_data = np.diff(diff_data)
    m2 = np.mean(diff_data**2)
    m4 = np.mean(diff_diff_data**2)
    return np.sqrt(m4 / m2)
    
# Update the main function to include these metrics
def eeg_to_csv_with_advanced_metrics(edf_file_path, channels, bands, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Prepare a list to collect the data
    all_data = []
    
    # For each channel
    for channel in channels:
        channel_data = {}
        channel_data['Channel'] = channel
        
        # Extract the EEG data for each frequency band and compute the metrics
        for band in bands:
            band_data = extract_data_by_band(edf_data, channel, band)
            
            # Basic metrics
            channel_data[f"{band}_mean"] = np.mean(band_data)
            channel_data[f"{band}_std"] = np.std(band_data)
            channel_data[f"{band}_variance"] = np.var(band_data)
            channel_data[f"{band}_zero_crossing_rate"] = zero_crossing_rate(band_data)
            channel_data[f"{band}_upper_margin"] = np.max(band_data)
            channel_data[f"{band}_lower_margin"] = np.min(band_data)
            channel_data[f"{band}_width"] = channel_data[f"{band}_upper_margin"] - channel_data[f"{band}_lower_margin"]
            channel_data[f"{band}_asymmetry"] = skew(band_data)
            channel_data[f"{band}_cv"] = np.std(band_data) / np.mean(band_data)  # Coefficient of variation
            channel_data[f"{band}_total_power"] = np.sum(band_data**2)
            channel_data[f"{band}_absolute_band_power"] = np.sum(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_mean_band_power"] = np.mean(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_relative_band_power"] = channel_data[f"{band}_absolute_band_power"] / channel_data[f"{band}_total_power"]
            channel_data[f"{band}_shannon_entropy"] = shannon_entropy(band_data)
            channel_data[f"{band}_katz_fd"] = katz_fd(band_data)
            channel_data[f"{band}_sample_entropy"] = sample_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_approximate_entropy"] = approximate_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_hjorth"] = hjorth_parameter(band_data)
            
        # Add the channel data to the main data list
        all_data.append(channel_data)
    # Convert the data list to a DataFrame and save it to a CSV file
    df = pd.DataFrame(all_data)
    df.to_csv(output_csv_path, index=False)
    return output_csv_path


# Extract data by band function
def extract_data_by_band(edf_data, channel_name, band_name):
    bands = {
        "delta": (2, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "low-beta": (13, 20)
    }
    l_freq, h_freq = bands[band_name]
    edf_data_filtered = edf_data.copy()
    edf_data_filtered.filter(l_freq=l_freq, h_freq=h_freq, picks=[channel_name], verbose=False)
    channel_data, _ = edf_data_filtered[edf_data_filtered.ch_names.index(channel_name)]
    return channel_data[0]

# Advanced metrics functions
def zero_crossing_rate(data):
    #"""Compute the zero-crossing rate of a signal."""
    return ((data[:-1] * data[1:]) < 0).sum() / len(data)

def shannon_entropy(data):
    #"""Compute the Shannon entropy of a signal."""
    p_data = data / data.sum()
    return entropy(p_data)

def katz_fd(data):
    #"""Compute Katz fractal dimension of a signal."""
    n = len(data)
    L = np.abs(data - data[0]).sum()
    d = np.abs(data - data[0]).max()
    return np.log10(n) / (np.log10(d/L) + np.log10(n))

def sample_entropy(data, m=2, r=0.2):
    #"""Compute the sample entropy of a signal."""
    N = len(data)
    A = np.sum([np.sum(np.abs(data[i:i+m] - data[j:j+m]) < r) for i in range(N - m) for j in range(i, N - m)]) - N - m + 1
    B = np.sum([np.sum(np.abs(data[i:i+m+1] - data[j:j+m+1]) < r) for i in range(N - m - 1) for j in range(i, N - m - 1)])
    return -np.log(A / B)

def approximate_entropy(data, m=2, r=0.2):
    #"""Compute the approximate entropy of a signal."""
    N = len(data)
    phi_m = np.sum([np.sum(np.abs(data[i:i+m] - data[j:j+m]) < r) for i in range(N - m) for j in range(i, N - m)]) / (N - m + 1)
    phi_m1 = np.sum([np.sum(np.abs(data[i:i+m+1] - data[j:j+m+1]) < r) for i in range(N - m - 1) for j in range(i, N - m - 1)]) / (N - m)
    return phi_m - phi_m1

def hjorth_parameter(data):
    #"""Compute the Hjorth parameter of a signal."""
    diff_data = np.diff(data)
    diff_diff_data = np.diff(diff_data)
    m2 = np.mean(diff_data**2)
    m4 = np.mean(diff_diff_data**2)
    return np.sqrt(m4 / m2)    


In [ ]:
def segment_into_microstates(data):
    """
    Segment the EEG data into microstates.
    For simplicity, this function will use a basic approach of clustering the EEG topographies.
    This is a basic approach and there are more advanced methods available.
    Returns a sequence of microstate assignments for each time point.
    """
    # For simplicity, we'll assume 4 microstate classes (A, B, C, D)
    # We'll assign each time point to one of these classes based on some basic criteria
    # (this is a placeholder approach; in real applications, more advanced clustering techniques are used)
    assignments = []
    mean_data = np.mean(data)
    for value in data:
        if value < 0.75 * mean_data:
            assignments.append('A')
        elif value < mean_data:
            assignments.append('B')
        elif value < 1.25 * mean_data:
            assignments.append('C')
        else:
            assignments.append('D')
    return assignments

# Integrate microstate features into the CSV
def integrate_microstate_features_to_csv(edf_file_path, channels, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Load existing CSV
    df = pd.read_csv(output_csv_path)
    
    # For each channel, compute microstate features
    for channel in channels:
        data, _ = edf_data[channel, :]
        data = data[0]
        
        # Segment data into microstates
        assignments = segment_into_microstates(data)
        
        # Extract microstate features
        durations, occurrences, contributions, mean_duration, mean_occurrence = extract_microstate_features(assignments)
        
        # Compute mean GFP
        gfp = np.std(data, axis=0)
        mean_gfps = compute_mean_gfp(data, assignments, gfp)
        
        # Append features to dataframe
        channel_idx = df[df['Channel'] == channel].index[0]
        df.at[channel_idx, 'mean_microstate_duration'] = mean_duration
        df.at[channel_idx, 'mean_microstate_occurrence'] = mean_occurrence
        df.at[channel_idx, 'mean_microstate_gfp_A'] = mean_gfps[0]
        df.at[channel_idx, 'mean_microstate_gfp_B'] = mean_gfps[1]
        df.at[channel_idx, 'mean_microstate_gfp_C'] = mean_gfps[2]
        df.at[channel_idx, 'mean_microstate_gfp_D'] = mean_gfps[3]
    
    # Save the updated dataframe to CSV
    df.to_csv(output_csv_path, index=False)
    return output_csv_path


In [12]:
def compute_mean_gfp_v2(data, assignments, gfp):
    """Compute mean GFP during each microstate class."""
    mean_gfps = []
    unique_states = np.unique(assignments)
    for state in unique_states:
        # Since GFP is now a single value, we don't need to average it over microstates
        mean_gfp = gfp
        mean_gfps.append(mean_gfp)
    return mean_gfps


def integrate_microstate_features_to_corrected_csv_v4(edf_file_path, channels, existing_csv_path, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Load existing CSV
    df = pd.read_csv(existing_csv_path)
    
    # For each channel, compute microstate features
    for channel in channels:
        data, _ = edf_data[channel, :]
        data = data[0]
        
        # Segment data into microstates
        assignments = segment_into_microstates(data)
        
        # Extract microstate features
        durations, occurrences, contributions, mean_duration, mean_occurrence = extract_microstate_features(assignments)
        
        # Compute mean GFP (removing axis specification for 1D data)
        gfp = np.std(data)
        mean_gfps = compute_mean_gfp_v2(data, assignments, gfp)
        
        # Append features to dataframe
        channel_idx = df[df['Channel'] == channel].index[0]
        df.at[channel_idx, 'mean_microstate_duration'] = mean_duration
        df.at[channel_idx, 'mean_microstate_occurrence'] = mean_occurrence
        
        # Dynamically assign GFP values to microstate classes
        unique_states = np.unique(assignments)
        for idx, state in enumerate(unique_states):
            df.at[channel_idx, f'mean_microstate_gfp_{state}'] = mean_gfps[idx]
    
    # Save the updated dataframe to CSV
    df.to_csv(output_csv_path, index=False)
    return output_csv_path
def revised_shannon_entropy(data, num_bins=100):
    """Compute the Shannon entropy of a signal's amplitude distribution."""
    # Compute histogram of the data
    hist, bin_edges = np.histogram(data, bins=num_bins, density=True)
    
    # Compute Shannon entropy of the histogram
    ent = -np.sum(hist * np.log2(hist + np.finfo(float).eps))  # Adding a small value to avoid log(0)
    
    return ent
# Retry integrating microstate features to the existing CSV with the corrected mean GFP computation

def optimized_sample_entropy(data, m=2, r=0.2):
    """Compute the sample entropy of a signal using a numba-optimized version."""
    N = len(data)
    A = 0
    B = 0
    
    # Compute A
    for i in range(N - m):
        for j in range(i, N - m):
            if np.max(np.abs(data[i:i+m] - data[j:j+m])) < r:
                A += 1
                
    # Adjust A
    A = A - (N - m + 1)
    
    # Compute B
    for i in range(N - m - 1):
        for j in range(i, N - m - 1):
            if np.max(np.abs(data[i:i+m+1] - data[j:j+m+1])) < r:
                B += 1
                
    return -np.log(A / B)

def optimized_approximate_entropy(data, m=2, r=0.2):
    """Compute the approximate entropy of a signal with optimized computation."""
    N = len(data)
    phi = np.zeros(2)
    
    for idx in range(2):
        m = m + idx
        run = np.zeros(N - m + 1)
        run[N - m] = 1
        
        for i in range(N - m - 1):
            tmp = np.abs(data[i:i+m] - data[i+1:i+m+1]) < r
            run[i] = run[i + 1] + tmp.all() - tmp[0]
        
        phi[idx] = np.mean(run) / (N - m + 1)
        
    return phi[0] - phi[1]

def segment_into_microstates(data):
    """
    Segment the EEG data into microstates.
    For simplicity, this function will use a basic approach of clustering the EEG topographies.
    This is a basic approach and there are more advanced methods available.
    Returns a sequence of microstate assignments for each time point.
    """
    # For simplicity, we'll assume 4 microstate classes (A, B, C, D)
    # We'll assign each time point to one of these classes based on some basic criteria
    # (this is a placeholder approach; in real applications, more advanced clustering techniques are used)
    assignments = []
    mean_data = np.mean(data)
    for value in data:
        if value < 0.75 * mean_data:
            assignments.append('A')
        elif value < mean_data:
            assignments.append('B')
        elif value < 1.25 * mean_data:
            assignments.append('C')
        else:
            assignments.append('D')
    return assignments

In [ ]:

# Integrate microstate features to the CSV
output_csv_with_microstates_path = "eeg_metrics_with_microstates.csv"
integrate_microstate_features_to_corrected_csv_v4("data/data_Subject00_1_6.edf", channels, output_csv_path, output_csv_with_microstates_path)
integrate_microstate_features_to_csv("data/data_Subject00_1_6.edf", channels, output_csv_with_microstates_path)


In [13]:
def eeg_to_csv_with_all_features(edf_file_path, channels, bands, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Prepare a list to collect the data
    all_data = []
    
    # For each channel
    for channel in channels:
        channel_data = {}
        channel_data['Channel'] = channel
        
        # Extract the EEG data for each frequency band and compute the metrics
        for band in bands:
            band_data = extract_data_by_band(edf_data, channel, band)
            
            # Basic metrics
            channel_data[f"{band}_mean"] = np.mean(band_data)
            channel_data[f"{band}_std"] = np.std(band_data)
            channel_data[f"{band}_variance"] = np.var(band_data)
            channel_data[f"{band}_zero_crossing_rate"] = zero_crossing_rate(band_data)
            channel_data[f"{band}_upper_margin"] = np.max(band_data)
            channel_data[f"{band}_lower_margin"] = np.min(band_data)
            channel_data[f"{band}_width"] = channel_data[f"{band}_upper_margin"] - channel_data[f"{band}_lower_margin"]
            channel_data[f"{band}_asymmetry"] = skew(band_data)
            channel_data[f"{band}_cv"] = np.std(band_data) / np.mean(band_data)  # Coefficient of variation
            channel_data[f"{band}_total_power"] = np.sum(band_data**2)
            channel_data[f"{band}_absolute_band_power"] = np.sum(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_mean_band_power"] = np.mean(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_relative_band_power"] = channel_data[f"{band}_absolute_band_power"] / channel_data[f"{band}_total_power"]
            channel_data[f"{band}_shannon_entropy"] = revised_shannon_entropy(band_data)
            channel_data[f"{band}_katz_fd"] = katz_fd(band_data)
            channel_data[f"{band}_sample_entropy"] = optimized_sample_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_approximate_entropy"] = optimized_approximate_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_hjorth"] = hjorth_parameter(band_data)
            
        # Microstate features
        data, _ = edf_data[channel, :]
        data = data[0]
        
        # Segment data into microstates
        assignments = segment_into_microstates(data)
        
        # Extract microstate features
        durations, occurrences, contributions, mean_duration, mean_occurrence = extract_microstate_features(assignments)
        
        # Compute mean GFP (removing axis specification for 1D data)
        gfp = np.std(data)
        mean_gfps = compute_mean_gfp_v2(data, assignments, gfp)
        
        # Add features to channel data
        channel_data['mean_microstate_duration'] = mean_duration
        channel_data['mean_microstate_occurrence'] = mean_occurrence
        unique_states = np.unique(assignments)
        for idx, state in enumerate(unique_states):
            channel_data[f'mean_microstate_gfp_{state}'] = mean_gfps[idx]
        
        # Add the channel data to the main data list
        all_data.append(channel_data)
    
    # Convert the data list to a DataFrame and save it to a CSV file
    df = pd.DataFrame(all_data)
    df.to_csv(output_csv_path, index=False)
    return output_csv_path
# List of channels and frequency bands
channels = ["Fp1", "Fp2", "F3", "F4", "F7", "F8", "Fz", "T7", "T8"]
bands = ["delta", "theta", "alpha", "low-beta"]
# Generate CSV with all features
output_csv_with_all_features_path = "data/eeg_all_features.csv"
eeg_to_csv_with_all_features("data/data_Subject00_1_6.edf", channels, bands, output_csv_with_all_features_path)


Extracting EDF parameters from /Users/sid/Documents/GitHub/EEG analysis/EEG-analysis/Dataset 4/data/data_Subject00_1_6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 999  =      0.000 ...     1.998 secs...


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12.0.
  this_h = firwin(this_N, (prev_freq + this_freq) / 2.,
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:313: DeprecationWarning: Keyword argument 'nyq' is deprecated in favour of 'fs' and will be removed in SciPy 1.12

'data/eeg_all_features.csv'

In [ ]:
def compute_gfp(data):
    """Compute the Global Field Power (GFP) of the EEG data."""
    return np.std(data, axis=0)

def segment_into_microstates_kmeans(data, n_clusters=4):
    """Segment EEG data into microstates using k-means clustering on GFP peaks."""
    gfp = compute_gfp(np.expand_dims(data, axis=0))
    
    # Identify GFP peaks
    gfp_peaks = (gfp[:-2] < gfp[1:-1]) & (gfp[1:-1] > gfp[2:])
    gfp_peak_indices = np.where(gfp_peaks)[0] + 1

    if len(gfp_peak_indices) == 0:
        return None, None  # No GFP peaks found

    # Extract topographies at GFP peaks
    topographies = np.expand_dims(data[gfp_peak_indices], axis=1)

    # Apply k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(topographies)
    
    # Assign each time point to a cluster based on similarity
    assignments = kmeans.predict(np.expand_dims(data, axis=1))

    return assignments, kmeans.cluster_centers_

def eeg_to_csv_with_all_features_kmeans(edf_file_path, channels, bands, output_csv_path):
    # Load the EEG data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Prepare a list to collect the data
    all_data = []
    
    # For each channel
    for channel in channels:
        channel_data = {}
        channel_data['Channel'] = channel
        
        # Extract the EEG data for each frequency band and compute the metrics
        for band in bands:
            band_data = extract_data_by_band(edf_data, channel, band)
            
            # Basic metrics
            channel_data[f"{band}_mean"] = np.mean(band_data)
            channel_data[f"{band}_std"] = np.std(band_data)
            channel_data[f"{band}_variance"] = np.var(band_data)
            channel_data[f"{band}_zero_crossing_rate"] = zero_crossing_rate(band_data)
            channel_data[f"{band}_upper_margin"] = np.max(band_data)
            channel_data[f"{band}_lower_margin"] = np.min(band_data)
            channel_data[f"{band}_width"] = channel_data[f"{band}_upper_margin"] - channel_data[f"{band}_lower_margin"]
            channel_data[f"{band}_asymmetry"] = skew(band_data)
            channel_data[f"{band}_cv"] = np.std(band_data) / np.mean(band_data)  # Coefficient of variation
            channel_data[f"{band}_total_power"] = np.sum(band_data**2)
            channel_data[f"{band}_absolute_band_power"] = np.sum(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_mean_band_power"] = np.mean(np.abs(np.fft.fft(band_data))**2)
            channel_data[f"{band}_relative_band_power"] = channel_data[f"{band}_absolute_band_power"] / channel_data[f"{band}_total_power"]
            channel_data[f"{band}_shannon_entropy"] = revised_shannon_entropy(band_data)
            channel_data[f"{band}_katz_fd"] = katz_fd(band_data)
            channel_data[f"{band}_sample_entropy"] = optimized_sample_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_approximate_entropy"] = optimized_approximate_entropy(band_data)  # m=2, r=0.2
            channel_data[f"{band}_hjorth"] = hjorth_parameter(band_data)
            
        # Microstate features using k-means clustering
        data, _ = edf_data[channel, :]
        data = data[0]
        
        # Segment data into microstates using k-means
        assignments, _ = segment_into_microstates_kmeans(data)
        
        # If assignments were found, extract microstate features
        if assignments is not None:
            durations, occurrences, contributions, mean_duration, mean_occurrence = extract_microstate_features(assignments)
            gfp = compute_gfp(data)
            mean_gfps = compute_mean_gfp(data, assignments, gfp)
        
            # Add features to channel data
            channel_data['mean_microstate_duration'] = mean_duration
            channel_data['mean_microstate_occurrence'] = mean_occurrence
        
            # Dynamically assign GFP values to microstate classes
            unique_states = np.unique(assignments)
            for idx, state in enumerate(unique_states):
                channel_data[f'mean_microstate_gfp_{state}'] = mean_gfps[idx]
        
            # Compute contributions
            for state in range(4):  # Assuming 4 clusters (A, B, C, D)
                channel_data[f'contribution_{state}'] = np.mean([gfp[i] for i in range(len(gfp)) if assignments[i] == state])
        else:
            # Set default values if no microstate assignments were found
            channel_data['mean_microstate_duration'] = np.nan
            channel_data['mean_microstate_occurrence'] = np.nan
            for state in range(4):
                channel_data[f'mean_microstate_gfp_{state}'] = np.nan
                channel_data[f'contribution_{state}'] = np.nan
        
        # Add the channel data to the main data list
        all_data.append(channel_data)
    
    # Convert the data list to a DataFrame and save it to a CSV file
    df = pd.DataFrame(all_data)
    df.to_csv(output_csv_path, index=False)
    return output_csv_path

# Retry generating the CSV with all features using k-means clustering for microstate segmentation
eeg_to_csv_with_all_features_kmeans("data/data_Subject00_1_6.edf", channels, bands, output_csv_with_all_features_path_kmeans)


# Not necessary anymore

In [3]:
# List of channels and frequency bands
channels = ["Fp1", "Fp2", "F3", "F4", "F7", "F8", "Fz", "T7", "T8"]
bands = ["delta", "theta", "alpha", "low-beta"]

# Test the function again
output_csv_path = "eeg_metrics.csv"
eeg_to_csv_with_advanced_metrics("data/data_Subject00_1_6.edf", channels, bands, output_csv_path)

Extracting EDF parameters from /Users/sid/Documents/GitHub/EEG analysis/EEG-analysis/Dataset 4/data/data_Subject00_1_6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 999  =      0.000 ...     1.998 secs...


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - %0.2g Hz'
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:1009: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logger.info('Setting up band-pass filter from %0.2g - %0.2g Hz'
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/mne/filter.py:1703: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a s

'eeg_metrics.csv'

# Old code

In [1]:
# Complete data for an epoch
edf_data = mne.io.read_raw_edf("data/data_Subject00_1_6.edf", preload=True)

NameError: name 'mne' is not defined

In [4]:
# Helper functions

def extract_data_by_band(edf_data, channel_name, band_name):
    bands = {
        "delta": (2, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "low-beta": (13, 20)
    }
    l_freq, h_freq = bands[band_name]
    edf_data_filtered = edf_data.copy()
    edf_data_filtered.filter(l_freq=l_freq, h_freq=h_freq, picks=[channel_name], verbose=False)
    channel_data, _ = edf_data_filtered[edf_data_filtered.ch_names.index(channel_name)]
    return channel_data[0]

def permutation_entropy(signal, order=3, delay=1, normalize=True):
    n = len(signal)
    L = []
    x = []
    for k in range(1, order + 1):
        Lk = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int((n - m) / k)):
                Lmk += abs(signal[m + i * k] - signal[m + i * k - k])
            Lmk = Lmk * (n - 1) / (int((n - m) / float(k)) * k)
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(1.0 / k), 1])
    (p, _, _, _) = np.linalg.lstsq(x, L, rcond=None)
    return p[0]

def higuchi_fd(data, k_max=10):
    n = len(data)
    L = []
    x = []
    for k in range(1, k_max + 1):
        Lk = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int((n - m) / k)):
                Lmk += abs(data[m + i * k] - data[m + i * k - k])
            Lmk = Lmk * (n - 1) / (int((n - m) / float(k)) * k)
            Lk.append(Lmk)
        L.append(np.log(np.mean(Lk)))
        x.append([np.log(1.0 / k), 1])
    (p, _, _, _) = np.linalg.lstsq(x, L, rcond=None)
    return p[0]

def katz_fd(data):
    n = len(data)
    L = np.hypot(np.diff(data), 1).sum()
    d = np.max(np.abs(data - data[0]))
    return np.log10(n) / (np.log10(d / L) + np.log10(n))

def modified_sample_entropy(U, m, r, epsilon=1e-10):
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for j in range(len(x)) if i != j and _maxdist(x[i], x[j]) <= r]) for i in range(len(x))]
        return sum(C)
    N = len(U)
    return -np.log((_phi(m + 1) + epsilon) / (_phi(m) + epsilon))

def approximate_entropy(U, m, r):
    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for j in range(len(x)) if i != j and max([abs(ua - va) for ua, va in zip(x[i], x[j])]) <= r]) for i in range(len(x))]
        return sum(C) / (N - m + 1.0)
    N = len(U)
    return _phi(m) - _phi(m + 1)

def hjorth_param(data):
    mu = np.mean(data)
    N = len(data)
    return np.sum([(x - mu)**2 for x in data]) / N


def compute_all_metrics(edf_file_path, channel_name, band_name):
    """
    Computes various metrics for a specific channel and frequency band from an EDF file.
    
    Parameters:
    - edf_file_path: Path to the EDF file
    - channel_name: Name of the channel to extract data from
    - band_name: Name of the frequency band ('delta', 'theta', 'alpha', 'low-beta')
    
    Returns:
    - Dictionary containing various metrics for the specified channel and frequency band
    """
    
    # Load the EDF data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Extract data for the specified channel and frequency band
    data = extract_data_by_band(edf_data, channel_name, band_name)
    
    # Compute basic statistics
    metrics = {
        "Mean": np.mean(data),
        "Variance": np.var(data),
        "Standard Deviation": np.std(data),
        "Skewness": skew(data),
        "Kurtosis": kurtosis(data),
        "Upper Margin": np.max(data),
        "Lower Margin": np.min(data),
        "Width": np.max(data) - np.min(data),
        "Asymmetry": np.mean(data) - np.median(data),
        "Coefficient of Variation": np.std(data) / np.mean(data) if np.mean(data) != 0 else 0,
        "Hjorth Parameter": hjorth_param(data),
        "Permutation Entropy": permutation_entropy(data),
        "Higuchi Fractal Dimension": higuchi_fd(data),
        "Katz Fractal Dimension": katz_fd(data)
    }
    
    # Parameters for entropy calculations
    m = 2  # Embedded dimension
    r = 0.2 * np.std(data)  # Tolerance
    
    # Compute other metrics
    metrics["Sample Entropy"] = modified_sample_entropy(data, m, r)
    metrics["Approximate Entropy"] = approximate_entropy(data, m, r)
    
    return metrics

def fit_microstates(data, maps):
    """Fit microstate sequence to EEG data."""
    # Assign each time point to the closest microstate map
    activations = np.dot(maps, data)
    assignments = np.argmax(activations, axis=0)
    return assignments

def extract_microstate_features(assignments):
    """Extract microstate features from assignments."""
    n_states = len(np.unique(assignments))
    durations = []  # Duration of each microstate
    occurrences = []  # Number of occurrences of each microstate
    for i in range(n_states):
        state_indices = np.where(assignments == i)[0]
        state_durations = np.diff(state_indices)
        durations.append(np.mean(state_durations))
        occurrences.append(len(state_durations))
    contributions = [d * o for d, o in zip(durations, occurrences)]  # Contribution of each microstate
    total_duration = sum(contributions)
    contributions = [c / total_duration for c in contributions]
    mean_duration = np.mean(durations)
    mean_occurrence = np.mean(occurrences)
    return durations, occurrences, contributions, mean_duration, mean_occurrence



In [ ]:
def compute_all_metrics_with_microstates_streamlined(edf_file_path, channel_name, band_name):
    """
    Computes various metrics, including microstate features, for a specific channel and frequency band from an EDF file.
    
    Parameters:
    - edf_file_path: Path to the EDF file
    - channel_name: Name of the channel to extract data from
    - band_name: Name of the frequency band ('delta', 'theta', 'alpha', 'low-beta')
    
    Returns:
    - Dictionary containing various metrics for the specified channel and frequency band
    """
    
    # Load the EDF data
    edf_data = mne.io.read_raw_edf(edf_file_path, preload=True)
    
    # Extract data for the specified channel and frequency band
    data = extract_data_by_band(edf_data, channel_name, band_name)
    
    # Compute basic statistics
    metrics = {
        "Mean": np.mean(data),
        "Variance": np.var(data),
        "Standard Deviation": np.std(data)
    }
    
    # Parameters for entropy calculations
    m = 2  # Embedded dimension
    r = 0.2 * np.std(data)  # Tolerance
    
    # Compute other metrics
    metrics["Sample Entropy"] = modified_sample_entropy(data, m, r)
    
    # Compute microstate metrics
    # Compute the Global Field Power (GFP)
    gfp = np.std(data, axis=0)
    
    # Define number of clusters (commonly 4 for microstate analysis)
    n_clusters = 4

    # Perform k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data.reshape(-1, 1))
    
    # Get the microstate maps (centroids)
    maps = kmeans.cluster_centers_

    # Fit microstate sequence to the EEG data
    activations = np.dot(maps, data.reshape(1, -1))  # Reshaping data to match matrix multiplication requirements
    assignments = np.argmax(activations, axis=0)
    
    # Extract microstate features
    durations, occurrences, contributions, mean_duration, mean_occurrence = extract_microstate_features(assignments)
    
    # Compute mean GFP during each microstate class
    mean_gfps = compute_mean_gfp(data, assignments, gfp)
    
    # Add microstate metrics to the main metrics dictionary
    metrics["Microstate Durations"] = durations
    metrics["Microstate Occurrences"] = occurrences
    metrics["Microstate Contributions"] = contributions
    metrics["Mean Microstate Duration"] = mean_duration
    metrics["Mean Microstate Occurrence"] = mean_occurrence
    metrics["Mean Microstate GFPs"] = mean_gfps
    
    return metrics

In [ ]:
def compute_mean_gfp_corrected(data, assignments, gfp):
    """Compute mean GFP during each microstate class with correction."""
    mean_gfps = []
    unique_states = np.unique(assignments)
    for state in unique_states:
        # No need to index into gfp as it's a scalar
        mean_gfp = gfp
        mean_gfps.append(mean_gfp)
    return mean_gfps

# Testing the corrected function
test_data = extract_data_by_band(mne.io.read_raw_edf("data/data_Subject00_1_6.edf", preload=True), "Fp1", "alpha")
test_gfp = np.std(test_data, axis=0)
test_kmeans = KMeans(n_clusters=4, random_state=0).fit(test_data.reshape(-1, 1))
test_maps = test_kmeans.cluster_centers_
test_activations = np.dot(test_maps, test_data.reshape(1, -1))
test_assignments = np.argmax(test_activations, axis=0)

compute_mean_gfp_corrected(test_data, test_assignments, test_gfp)

In [ ]:
# List of channels and frequency bands
channels = ["Fp1", "Fp2", "F3", "F4", "F7", "F8", "Fz", "T3", "T4"]
bands = ["delta", "theta", "alpha", "low-beta"]

# Processing the provided EDF file for each channel and frequency band
results = []

# Re-running the optimized approach to compute metrics for each channel and frequency band
df_results_optimized = pd.DataFrame()

for channel in channels:
    channel_results = []
    for band in bands:
        metrics = compute_all_metrics_with_microstates_streamlined("data/data_Subject00_1_6.edf", channel, band)
        metrics["Channel"] = channel
        metrics["Frequency Band"] = band
        channel_results.append(metrics)
    df_channel = pd.DataFrame(channel_results)
    df_results_optimized = df_results_optimized.append(df_channel, ignore_index=True)

csv_path_optimized = "results_Subject00_1_6_optimized.csv"
df_results_optimized.to_csv(csv_path_optimized, index=False)

csv_path_optimized